In [129]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.base import BaseEstimator, TransformerMixin

from flask import Blueprint, request, jsonify, render_template, Flask

import requests
import json
from flask import Blueprint, request, jsonify, render_template, Flask
from flask_cors import CORS, cross_origin
import pickle
import os

In [130]:
class My_encoder(BaseEstimator, TransformerMixin):
   
    def __init__(self,drop = 'first',sparse=False):
        self.encoder = OneHotEncoder(drop = drop,sparse = sparse)
        self.features_to_encode = []
        self.columns = []
    
    def fit(self,X_train,features_to_encode):
        
        data = X_train.copy()
        self.features_to_encode = features_to_encode
        data_to_encode = data[self.features_to_encode]
        self.columns = pd.get_dummies(data_to_encode,drop_first = True).columns
        self.encoder.fit(data_to_encode)
        return self.encoder
    
    def transform(self,X_test):
        
        data = X_test.copy()
        data.reset_index(drop = True,inplace =True)
        data_to_encode = data[self.features_to_encode]
        data_left = data.drop(self.features_to_encode,axis = 1)
        
        data_encoded = pd.DataFrame(self.encoder.transform(data_to_encode),columns = self.columns)
        
        return pd.concat([data_left,data_encoded],axis = 1)

In [138]:
#################################################
# Activating CORS
app = Flask(__name__)
app.config['CORS_HEADERS'] = 'Content-Type'
cors = CORS(app)
#################################################

In [ ]:
def safe_paths():
    '''
    Function helper to define path of pickled model and dataframes.
    '''
    
    model_path = os.path.join('model_pickle.pkl')
    train_path = os.path.join('df_encoded_for_train.pkl')
    ref_path = os.path.join('df_encoded_for_ref.pkl')
    
    return (model_path, train_path, ref_path)


def load_models():
    '''
    Helper function to load pickled model and preprocessed dataframes
    '''
    model_path, train_path, ref_path = safe_paths()
    
    pickled_model = pickle.load(open(model_path, 'rb'))
    pickled_train = pickle.load(open(train_path, 'rb'))
    pickled_ref = pickle.load(open(ref_path, 'rb'))
        
    return (pickled_model, pickled_train, pickled_ref)  


############################################################
'Saving loaded pickled files in global memory'

PICKLED_MODEL, PICKLED_TRAIN, PICKLED_REF = load_models()
#############################################################


def recommend_track_id(track_id = '2YegxR5As7BeQuVp2U6pek'):
    '''
    Helper function that takes in a track_id and recommends 14 track_ids of similar songs
    '''
    
    song_num = PICKLED_REF[PICKLED_REF['track_id']==track_id].song_num.to_numpy()[0]
    distance, neighbors = PICKLED_MODEL.kneighbors(np.array([PICKLED_TRAIN.values[song_num]]))
    

    song_list = []
    for item in neighbors[0][1:]: # this way excludes itself
        row = PICKLED_REF.iloc[item]
        song_list.append((row.track_id))  # print list of track id
              
    return song_list

In [152]:
a= recommend_track_id('5PntSbMHC1ud6Vvl8x56qd')
a

['5z6xHjCZr7a7AIcy8sPBKy',
 '0BIIVWkv7nRdYKCNxQUbfh',
 '3HE50TVRquwXe9yv2HFoNL',
 '7jp3STyCSsoakSPUoKTtFX',
 '56LfnhlS3xvjtCpXWOxleW',
 '5j0oqJkeHoXjtPCgz9hMmd',
 '3i8m18jJ3yc2WNT0HxBtsK',
 '2lwwrWVKdf3LR9lbbhnr6R',
 '5VJRSXqHca3yAsCEISymlc',
 '77LywUODoUWlQJxsKQBpHw',
 '0nkzd3yNniB767zSDDdLZ3',
 '45LF89wEgE97hGYPM2doJN',
 '0sp00HSXkQyqTa6QqM0O8V',
 '5K5OS3f4aBjwpbnfQ9ww1r']

In [186]:
adf= pd.DataFrame(a, columns=['recs'])
adf

,recs
0,5z6xHjCZr7a7AIcy8sPBKy
1,0BIIVWkv7nRdYKCNxQUbfh
2,3HE50TVRquwXe9yv2HFoNL
3,7jp3STyCSsoakSPUoKTtFX
4,56LfnhlS3xvjtCpXWOxleW
5,5j0oqJkeHoXjtPCgz9hMmd
6,3i8m18jJ3yc2WNT0HxBtsK
7,2lwwrWVKdf3LR9lbbhnr6R
8,5VJRSXqHca3yAsCEISymlc
9,77LywUODoUWlQJxsKQBpHw


In [169]:
b =adf.to_json()

In [178]:
type(b)

str

In [183]:
b.title()

'{"Recs":{"0":"5Z6Xhjczr7A7Aicy8Spbky","1":"0Biivwkv7Nrdykcnxqubfh","2":"3He50Tvrquwxe9Yv2Hfonl","3":"7Jp3Stycssoakspuokttfx","4":"56Lfnhls3Xvjtcpxwoxlew","5":"5J0Oqjkehoxjtpcgz9Hmmd","6":"3I8M18Jj3Yc2Wnt0Hxbtsk","7":"2Lwwrwvkdf3Lr9Lbbhnr6R","8":"5Vjrsxqhca3Yasceisymlc","9":"77Lywuodouwlqjxskqbphw","10":"0Nkzd3Ynnib767Zsdddlz3","11":"45Lf89Wege97Hgypm2Dojn","12":"0Sp00Hsxkqyqta6Qqm0O8V","13":"5K5Os3F4Abjwpbnfq9Ww1R"}}'

In [184]:
dir(b)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',
